# Weather Data

In this notebook, we import the weather data, process it and join it on the trips.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import sys, os
sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2

from modules.config import WEATHER_AGGR_TEMPORAL_PATH

In [3]:
# Station ID for Leipzig-Holzhausen 
# https://www.dwd.de/DE/leistungen/klimadatendeutschland/statliste/statlex_html.html?view=nasPublication&nn=16102

# data air temperature
# https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/historical/stundenwerte_TU_02928_19510101_20211231_hist.zip
df_temp = pd.read_csv('../00_data/repo_data/weather/stundenwerte_TU_02928_19510101_20211231_hist/produkt_tu_stunde_19510101_20211231_02928.txt', delimiter=";")

# data precipiation
# https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/precipitation/historical/stundenwerte_RR_02928_19970407_20211231_hist.zip
df_precip = pd.read_csv('../00_data/repo_data/weather/stundenwerte_RR_02928_19970407_20211231_hist/produkt_rr_stunde_19970407_20211231_02928.txt', delimiter=";")

# data wind
# https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/wind/historical/stundenwerte_FF_02928_19580101_20211231_hist.zip
df_wind = pd.read_csv('../00_data/repo_data/weather/stundenwerte_FF_02928_19580101_20211231_hist/produkt_ff_stunde_19580101_20211231_02928.txt', delimiter = ";")

# data cloud cover
# https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/cloudiness/historical/stundenwerte_N_02928_19970407_20211231_hist.zip
df_cloud = pd.read_csv('../00_data/repo_data/weather/stundenwerte_N_02928_19970407_20211231_hist/produkt_n_stunde_19970407_20211231_02928.txt', delimiter = ";")


Clean the data:
- remove unnecessary features
- restrict time frame to 2019
- convert 'MESS_DATUM' feature to timestamp and set as index

In [4]:
df_temp.drop(['STATIONS_ID','eor', 'QN_9', 'RF_TU'], inplace=True, axis=1)
# description of columns
# https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/air_temperature/historical/DESCRIPTION_obsgermany_climate_hourly_tu_historical_en.pdf
df_temp.rename(columns = {'TT_TU':'temperature'},inplace = True)

df_temp = df_temp[df_temp['MESS_DATUM'].between(2019000000, 2019311223)]

df_temp['MESS_DATUM'] = list([pd.to_datetime(x, format='%Y%m%d%H') for x in df_temp['MESS_DATUM'].to_list()])
df_temp.set_index('MESS_DATUM', inplace=True)

In [5]:
df_precip.drop(['STATIONS_ID','eor', 'QN_8','  R1','WRTR'], inplace=True, axis=1)
# description of columns
# https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/precipitation/historical/DESCRIPTION_obsgermany_climate_hourly_precipitation_historical_en.pdf
df_precip.rename(columns = {'RS_IND':'precipitation'}, inplace = True)

df_precip = df_precip[df_precip['MESS_DATUM'].between(2019000000, 2019311223)]

df_precip['MESS_DATUM'] = list([pd.to_datetime(x, format='%Y%m%d%H') for x in df_precip['MESS_DATUM'].to_list()])
df_precip.set_index('MESS_DATUM', inplace=True)

In [6]:
df_wind.drop(['STATIONS_ID','eor','QN_3', '   D'], inplace=True, axis=1)
# description of columns
# https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/wind/historical/DESCRIPTION_obsgermany_climate_hourly_wind_historical_en.pdf
df_wind.rename(columns = {'   F':'mean_wind_speed'}, inplace = True)

df_wind = df_wind[df_wind['MESS_DATUM'].between(2019000000, 2019311223)]

df_wind['MESS_DATUM'] = list([pd.to_datetime(x, format='%Y%m%d%H') for x in df_wind['MESS_DATUM'].to_list()])
df_wind.set_index('MESS_DATUM', inplace=True)

In [7]:
df_cloud.drop(['STATIONS_ID','eor', 'QN_8', 'V_N_I'], inplace=True, axis=1)
# description of columns
# https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/cloudiness/historical/DESCRIPTION_obsgermany_climate_hourly_cloudiness_historical_en.pdf
df_cloud.rename(columns = {' V_N':'total_cloud_cover'}, inplace = True)

df_cloud = df_cloud[df_cloud['MESS_DATUM'].between(2019000000, 2019311223)]

df_cloud['MESS_DATUM'] = list([pd.to_datetime(x, format='%Y%m%d%H') for x in df_cloud['MESS_DATUM'].to_list()])
df_cloud.set_index('MESS_DATUM', inplace=True)

Create dataframe with every hour in 2019

In [8]:
hour_data = {'MESS_DATUM': pd.date_range('2019-01-01T00', '2019-12-31T23', freq='H')}
time_range = pd.DataFrame(hour_data)
time_range.set_index('MESS_DATUM', inplace=True)
time_range.head()

""
MESS_DATUM
2019-01-01 00:00:00
2019-01-01 01:00:00
2019-01-01 02:00:00
2019-01-01 03:00:00
2019-01-01 04:00:00


Merge weather data into hour dataframe

In [9]:
df_weather = time_range.join(df_temp).join(df_wind).join(df_cloud).join(df_precip)

df_weather

,temperature,mean_wind_speed,total_cloud_cover,precipitation
MESS_DATUM,,,,
2019-01-01 00:00:00,7.4,2.8,8.0,0.0
2019-01-01 01:00:00,7.7,2.9,8.0,0.0
2019-01-01 02:00:00,7.7,3.7,8.0,0.0
2019-01-01 03:00:00,7.7,3.2,8.0,0.0
2019-01-01 04:00:00,7.6,3.3,8.0,0.0
...,...,...,...,...
2019-12-31 19:00:00,2.8,2.3,1.0,0.0
2019-12-31 20:00:00,2.4,2.0,5.0,0.0
2019-12-31 21:00:00,2.7,2.3,7.0,0.0


Check for missing, null or inconsistent data

In [10]:
df_weather.isna().sum()

temperature            0
mean_wind_speed      153
total_cloud_cover    443
precipitation         73
dtype: int64

In [11]:
df_weather.describe()

,temperature,mean_wind_speed,total_cloud_cover,precipitation
count,8760.000000,8607.000000,8317.000000,8687.000000
mean,-0.747386,1.799268,5.601539,-4.528606
std,109.713974,24.162461,3.180506,68.487075
min,-999.000000,-999.000000,0.000000,-999.000000
25%,4.900000,1.400000,3.000000,0.000000
50%,10.100000,2.200000,7.000000,0.000000
75%,16.900000,3.100000,8.000000,0.000000
max,37.800000,10.100000,8.000000,1.000000


Replace -999 with NaN values for interpolation

In [12]:
df_weather.replace(-999, np.NaN, inplace= True)

In [13]:
df_weather.isna().sum()

temperature          104
mean_wind_speed      158
total_cloud_cover    443
precipitation        114
dtype: int64

Interpolate missing data

In [14]:
df_weather.interpolate(method="time", inplace=True)

# round interpolated values so they match the 0/1 or 0-8 value set
df_weather['precipitation'] = df_weather['precipitation'].round()
df_weather['total_cloud_cover'] = df_weather['total_cloud_cover'].round()

Aggregate the weather data with different time-intervals

In [15]:
def aggregate(weather_data):

    time_intervals = [1,2,6,24]
    data_1 = pd.DataFrame()
    data_2 = pd.DataFrame()
    data_6 = pd.DataFrame()
    data_24 = pd.DataFrame()

    for i in time_intervals:
        df = weather_data.copy()
        max_data = df.resample(f"{i}H").max().rename(columns = {'temperature': 'max_temperature', 'mean_wind_speed' : 'max_mean_wind_speed', 'precipitation' : 'max_precipitation','total_cloud_cover' : 'max_total_cloud_cover'})
        min_data = df.resample(f"{i}H").min().rename(columns = {'temperature': 'min_temperature', 'mean_wind_speed' : 'min_mean_wind_speed', 'precipitation' : 'min_precipitation', 'total_cloud_cover' : 'min_total_cloud_cover'})
        mean_data = df.resample(f"{i}H").mean().rename(columns = {'temperature': 'mean_temperature', 'mean_wind_speed' : 'mean_mean_wind_speed', 'precipitation' : 'mean_precipitation','total_cloud_cover' : 'mean_total_cloud_cover'})
        sum_data = df.resample(f"{i}H").sum().rename(columns = {'temperature': 'sum_temperature', 'mean_wind_speed' : 'sum_mean_wind_speed', 'precipitation' : 'sum_precipitation','total_cloud_cover' : 'sum_total_cloud_cover'})

        if(i == 1):
            data_1 = pd.concat([min_data, max_data, mean_data, sum_data], axis=1)
            data_1['time_interval_length'] = i
            data_1.reset_index()
            continue

        if(i == 2):
            data_2 = pd.concat([min_data, max_data, mean_data, sum_data], axis=1)
            data_2['time_interval_length'] = i
            data_2.reset_index()
            continue

        if(i == 6):
            data_6 = pd.concat([min_data, max_data, mean_data, sum_data], axis=1)
            data_6['time_interval_length'] = i
            data_6.reset_index()
            continue

        if(i == 24):
            data_24 = pd.concat([min_data, max_data, mean_data, sum_data], axis=1)
            data_24['time_interval_length'] = i
            data_24.reset_index()

        return pd.concat([data_1, data_2, data_6, data_24], axis=0)

In [16]:
agg_weather = aggregate(df_weather)
#print(agg_weather. columns)

agg_weather = agg_weather[['min_temperature','max_temperature', 'mean_temperature', 'mean_mean_wind_speed', 'mean_total_cloud_cover', 'sum_precipitation', 'time_interval_length']]

agg_weather

,min_temperature,max_temperature,mean_temperature,mean_mean_wind_speed,mean_total_cloud_cover,sum_precipitation,time_interval_length
MESS_DATUM,,,,,,,
2019-01-01 00:00:00,7.4,7.4,7.400000,2.800000,8.000000,0.0,1
2019-01-01 01:00:00,7.7,7.7,7.700000,2.900000,8.000000,0.0,1
2019-01-01 02:00:00,7.7,7.7,7.700000,3.700000,8.000000,0.0,1
2019-01-01 03:00:00,7.7,7.7,7.700000,3.200000,8.000000,0.0,1
2019-01-01 04:00:00,7.6,7.6,7.600000,3.300000,8.000000,0.0,1
...,...,...,...,...,...,...,...
2019-12-27 00:00:00,0.5,4.3,2.483333,1.591667,7.666667,8.0,24
2019-12-28 00:00:00,-3.6,1.6,-0.008333,1.291667,7.750000,1.0,24
2019-12-29 00:00:00,-4.0,2.5,-0.725000,1.766667,7.041667,0.0,24


Export as parquet

In [17]:
agg_weather.to_parquet(WEATHER_AGGR_TEMPORAL_PATH)